In [1]:
import torch
import matplotlib.pyplot as plt
from stlcgpp.formula import *
from stlcgpp.tests import *
from stlcgpp.viz import *

from matplotlib import rc
rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)


## NOTE
If using Expressions to define formulas, `stlcgpp` expects input signals to be of size `[time_dim]`.
If using Predicates to define formulas, `stlcgpp` expects input signals to be of size `[time_dim, state_dim]` where `state_dim` is the expected input size of your predicate function.



In [2]:
def compute_distance_to_origin(states):
    return torch.norm(states[...,:2], dim=-1, keepdim=False)

In [3]:
T = 10
compute_distance_to_origin(torch.ones([T, 2]))

tensor([1.4142, 1.4142, 1.4142, 1.4142, 1.4142, 1.4142, 1.4142, 1.4142, 1.4142,
        1.4142])

## Using Expressions
Expressions are placeholders for input signals. Specifically, it is assuming the signal is already a 1D array, such as the output of a predicate function. 

This is useful if you have signals from predicates computed already. 

In general, this is useful for readability and visualization.

In [4]:
distance_to_origin_exp = Expression("magnitude", value=None) # can define an Expression without setting values for the expression right now
formula_exp = Eventually(distance_to_origin_exp < 0.5) # can define an STL formula given an expression, again, the value of the expression does not need to be set yet


formula_exp(distance_to_origin_exp) # <---- this will throw an error since the expression does not have values set yet



AssertionError: Input Expression does not have numerical values

In [ ]:
# so let's go ahead and set a value for the expression
T = 5
states = torch.randn([T, 2])
states_norm = compute_distance_to_origin(states)   # compute distance to origin

distance_to_origin_exp.set_value(states_norm)   # set value for Expression

# compute robustness trace
formula_exp(distance_to_origin_exp) # <---- this will no longer throw an error since the expression has a value set

# alternatively, we can directly plug any torch.tensor and evaluate the robustness without 
states2 = torch.randn([T, 2])
states_norm2 = compute_distance_to_origin(states2)   # compute distance to origin
formula_exp(states_norm2) 



We can compute the robustness value (instead of trace) and take the derivative

In [ ]:
robustness = formula_exp.robustness(states_norm) 
print(f"Robustness value: {robustness:.3f}\n")

gradient = torch.func.grad(formula_exp.robustness)(states_norm) 
print(f"Gradient of robustness value w.r.t. input:\n {gradient}")


We can apply a smooth max/min approximation by selecting a `approx_method` and `temperature`.
The default `approx_method` is `true`.


In [ ]:
approx_method = "logsumexp"  # or "softmax"
temperature = 1. # needs to be > 0

robustness = formula_exp.robustness(states_norm, approx_method=approx_method, temperature=temperature) 
print(f"Robustness value: {robustness:.3f}\n")

gradient = torch.func.grad(formula_exp.robustness)(states_norm, approx_method=approx_method, temperature=temperature) 
print(f"Gradient of robustness value w.r.t. input:\n {gradient}") # <----- gradients are spread across different values

For formulas that are defined with two different Expressions, we need to be careful about the signals we are feeding in.

In [ ]:
# if both subformulas use the same signal, then we can do this
phi = (distance_to_origin_exp > 0) & (distance_to_origin_exp < 0.5)  
phi(states_norm)


# if the formula depends on two different signals, then we need to provide the two signals as tuple
distance_to_origin_exp = Expression("magnitude", value=None)
speed_exp = Expression("speed", value=None)

phi = (distance_to_origin_exp > 0) & (speed_exp < 0.5)  

phi(states_norm) # <--- Will give WRONG ANSWER


speed = torch.randn([T])
input_correct_order = (states_norm, speed)
input_wrong_order = (speed, states_norm)
phi(input_correct_order) # <--- Will give desired answer
phi(input_wrong_order) # <--- Will give WRONG ANSWER since the ordering of the input does not correspond to how phi is defined




## Using Predicates
Predicates are the functions that an N-D signal is passed through and its outputs are then passed through each operation of the STL formula.
We can construct an STL formula by specifying the predicate functions and the connectives and temporal operations.


In [ ]:
distance_to_origin_pred = Predicate("magnitude", predicate_function=compute_distance_to_origin) # define a predicate function with a name and the function
formula_pred = Eventually(distance_to_origin_pred < 0.5) # define the STL formula

# so let's go ahead and set a value for the input N-D array which will be the input into the predicate function.
T = 5
states = torch.randn([T, 2])  # 2D signal
output_from_using_predicate = formula_pred(states) # compute distance to origin INSIDE 


# NOTE: this is equivalent to the following with expressions
states_norm = compute_distance_to_origin(states)   # computes distance to origin OUTSIDE 
output_from_using_expression = formula_exp(states_norm) 


# check if we get the same answer
torch.isclose(output_from_using_predicate, output_from_using_expression)

Similarly, we can compute the robustness value (instead of trace) and take the derivative. 

In [ ]:
approx_method = "logsumexp"  # or "softmax"
temperature = 1. # needs to be > 0

robustness = formula_pred.robustness(states, approx_method=approx_method, temperature=temperature) 
print(f"Robustness value: {robustness:.3f}\n")

gradient = torch.func.grad(formula_pred.robustness)(states, approx_method=approx_method, temperature=temperature) 
print(f"Gradient of robustness value w.r.t. input:\n {gradient}") # <----- gradients are spread across different values

Note that when taken gradients with formulas defined with predicates, the input is the N-D signal which is passed into the predicate function and other robustness formulas. That is to say, the gradient will be influenced by the choice of the predicate. 

To get the same gradient output when using Expressions, we need to do the following:

In [ ]:
def foo(states):
    states_norm = compute_distance_to_origin(states)   # compute distance to origin
    return formula_exp.robustness(states_norm, approx_method=approx_method, temperature=temperature) 

torch.func.grad(foo)(states)

## Comparing the masking and recurrent approach to computing SRL robustness
The main difference between `STLCG` and `STLCG++` is that `STLCG` relies on recurrent computations to compute robustness and `STLCG++` used a masking approach. More details are provided in the accompanying paper.

Next, we show some examples on how call the recurrent implementation.

In [ ]:
predicate = Predicate("identity", lambda x: x)
recurrent = AlwaysRecurrent(predicate > 0.)
mask = Always(predicate > 0.)

# recurrent = UntilRecurrent(predicate > 0., predicate < 5.)
# mask = Until(predicate > 0., predicate < 5.)

T = 10
signal =  torch.arange(T).float()
signal


In [ ]:
# robustness trace using the masking approach.
# the values are exactly what we expect.
mask(signal), mask.robustness(signal)

To compute the robustness trace using the recurrent method, we need to input the signal **backward** in time

(In the future, will handling the reversing of signal internally, and user does not need to deal with it.)

In [ ]:
signal_flip = signal.flip(0)
recurrent(signal_flip).flip(0), recurrent.robustness(signal_flip)

### We can visualize the STL formulas!

In [ ]:
print(mask)

In [ ]:
make_stl_graph(mask)

In [ ]:
make_stl_graph(recurrent)